In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

from utils import processed, load, denormalize, get_normalized_adj, generate_dataset

In [2]:
class Temporal_Layer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel):
        super(Temporal_Layer, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, (1, kernel))
        self.conv2 = nn.Conv2d(in_channels, out_channels, (1, kernel))
        
    def forward(self, x):
        x = x.permute(0,3,1,2).type(torch.cuda.FloatTensor)
        normal = self.conv1(x)
        sig = torch.sigmoid(self.conv2(x))
        out = normal * sig
        out = out.permute(0,2,3,1)
        # Convert back from NCHW to NHWC
        return out
        
        
class Stgcn_Block(nn.Module):
    def __init__(self, in_channels, spatial_channels, out_channels, nodes_num):
        super(Stgcn_Block, self).__init__()
        self.temporal_layer1 = Temporal_Layer(in_channels, out_channels, kernel = 2) 
        self.temporal_layer2 = Temporal_Layer(in_channels = spatial_channels, out_channels = out_channels, kernel = 2)
        
        self.weight = nn.Parameter(torch.FloatTensor(out_channels, spatial_channels)) 
        self.initialise_weight()
        
        self.batch_norm = nn.BatchNorm2d(nodes_num)
        
        
    def initialise_weight(self):
        std_dv = 1. / math.sqrt(self.weight.shape[1])
        self.weight.data.uniform_(-std_dv, std_dv)
        
    def forward(self, x, adj_hat):
        # First temporal Block
        temporal_block1 = self.temporal_layer1(x)
        
        #Spatial Graph Convolution
        t = temporal_block1.permute(1,0,2,3) #Converts tensor from nhwc to hnwc for multiplication with adj_matrix
        t = t.type(torch.cuda.DoubleTensor)
        gconv1 = torch.einsum("ij, jklm -> kilm", [adj_hat, t]) #(h,h) * (h,n,w,c) -> (n,h,w,c)
        gconv2 = F.relu(torch.matmul(gconv1, self.weight.double()))
        
        #Second Temporal Block
        temporal_block2 = self.temporal_layer2(gconv2) 
        
        out = self.batch_norm(temporal_block2)
        return out

class Stgcn_Model(nn.Module):
    def __init__(self, nodes_num, features_num, input_timesteps, num_output):
        super(Stgcn_Model, self).__init__()
        self.stgcn_block1 = Stgcn_Block(in_channels = features_num, spatial_channels = 16, out_channels = 64,
                                       nodes_num = nodes_num)
        
        self.stgcn_block2 = Stgcn_Block(in_channels = 64, spatial_channels = 16,  out_channels = 64,
                                       nodes_num = nodes_num)
        
        self.temporal_layer = Temporal_Layer(in_channels = 64, out_channels = 64, kernel = 2)
        self.fc = nn.Linear((input_timesteps * 106) * 64, num_output)

    def forward(self, adj_hat, x):
        out1 = self.stgcn_block1(x, adj_hat)
        out2 = self.stgcn_block2(out1, adj_hat)
        out3 = self.temporal_layer(out2)
        #out3_temp = out3.reshape((out3.shape[0], out3.shape[1], -1)) #reshaped to torch.Size([12, 53, 64])
        out3_temp = out3.view(-1,40704)
        out4 = self.fc(out3_temp) 
        return out4

In [3]:
def train(x_input, x_target, batch_size):
    num_samples = x_input.shape[0]
    shuffled_order = torch.randperm(num_samples)
    
    training_loss = []
    for i in range(math.ceil(num_samples / batch_size)):
        stgcn.train()
        optimizer.zero_grad()
        
        start = i * batch_size
        batch = shuffled_order[start:start+batch_size]
        
        #TODO - think about how to avoid FC layer from crashing
        if len(batch) % batch_size != 0:
            continue
        
        x_batch = x_input[batch].to(device = device)
        y_batch = x_target[batch].to(device = device)
        
        
        out = stgcn(adj_mat, x_batch)
        loss = loss_criterion(out, y_batch)
        loss.backward()
        optimizer.step()
        
        training_loss.append(loss.detach().cpu().numpy())
        
    return sum(training_loss) / len(training_loss)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 50
batch_size = 12

num_timesteps_input = 6 # 30 minutes
num_timesteps_output = 2 # 10 minutes

In [5]:
raw_trunc_dir = "./data/raw/trunc/"
process_dir = "./data/processed/"

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
processed(raw_trunc_dir, process_dir, overwrite=False)
A, X, metadata, means, stds = load(process_dir)

split_line1 = int(X.shape[2] * 0.6)
split_line2 = int(X.shape[2] * 0.8)

train_original_data = X[:, :, :split_line1]
val_original_data = X[:, :, split_line1:split_line2]
test_original_data = X[:, :, split_line2:]

training_input, training_target = generate_dataset(train_original_data,
                                                   num_timesteps_input=num_timesteps_input,
                                                   num_timesteps_output=num_timesteps_output)
val_input, val_target = generate_dataset(val_original_data,
                                         num_timesteps_input=num_timesteps_input,
                                         num_timesteps_output=num_timesteps_output)
test_input, test_target = generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
adj_mat = get_normalized_adj(A)
adj_mat = torch.from_numpy(adj_mat).to(device)

In [6]:
stgcn = Stgcn_Model(nodes_num = adj_mat.shape[0], features_num = training_input.shape[3],
                    input_timesteps = num_timesteps_input, num_output = num_timesteps_output).to(device = device)

optimizer = torch.optim.Adam(stgcn.parameters(), lr = 1e-3)
loss_criterion = nn.MSELoss()

training_loss = []
validation_loss = []

for epoch in range(epochs):
    loss = train(training_input, training_target, batch_size)
    training_loss.append(loss)

    with torch.no_grad():
        stgcn.eval()
        val_input = val_input.to(device= device)
        val_target = val_target.to(device = device)

        out = stgcn(adj_mat, val_input)
        val_loss = loss_criterion(out, val_target).to(device = 'cpu')
        validation_loss.append(val_loss.item())

    print("Training Loss: {}".format(loss))
    print("Validation Loss: {}".format(val_loss))


plt.plot(training_loss, label = 'Training Loss')
plt.plot(validation_loss, label = 'Validation Loss')
plt.legend()
plt.show()


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([12, 53, 2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([1492, 53, 6, 3])


RuntimeError: shape '[-1, 40704]' is invalid for input of size 5060864